In [1]:
#import packages and the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
observations = pd.read_csv('https://raw.githubusercontent.com/noo-rashbass/synthetic-data-service/master/Aisha/observations.csv')

In [12]:
table2 = observations
bins=[0,37,38,39,40,np.inf]
names=['<37','37-38','38-39','39-40','40+']
table2['Temperature_range']=pd.cut(table2['Temperature (C) [EUPATH_0000110]'],bins,labels=names)

In [13]:
table2

,Observation_Id,Participant_Id,Household_Id,Abdominal pain [HP_0002027],Abdominal pain duration (days) [EUPATH_0000154],Admitting hospital [EUPATH_0000318],Age at visit (years) [EUPATH_0000113],Anorexia [SYMP_0000523],Anorexia duration (days) [EUPATH_0000155],"Asexual Plasmodium parasite density, by microscopy [EUPATH_0000092]",...,Severe malaria criteria [EUPATH_0000046],Subjective fever [EUPATH_0000100],"Submicroscopic Plasmodium present, by LAMP [EUPATH_0000487]",Temperature (C) [EUPATH_0000110],Visit date [EUPATH_0000091],Visit type [EUPATH_0000311],Vomiting [HP_0002013],Vomiting duration (days) [EUPATH_0000165],Weight (kg) [EUPATH_0000732],Temperature_range
0,100118961,1001,HH216001613,No,0.0,NaN,34.08,No,0.0,NaN,...,NaN,No,NaN,36.9,2011/11/27,Unscheduled visit,No,0.0,67.0,<37
1,100120115,1001,HH216001613,No,0.0,NaN,37.24,No,0.0,0.0,...,NaN,No,NaN,36.7,2015/1/24,Scheduled visit,No,0.0,72.0,<37
2,100120199,1001,HH216001613,No,0.0,NaN,37.47,No,0.0,0.0,...,NaN,No,NaN,37.0,2015/4/18,Scheduled visit,No,0.0,72.0,<37
3,100120031,1001,HH216001613,No,0.0,NaN,37.01,No,0.0,0.0,...,NaN,No,NaN,36.9,2014/11/1,Scheduled visit,No,0.0,73.0,<37
4,100119668,1001,HH216001613,No,0.0,NaN,36.02,No,0.0,0.0,...,NaN,No,NaN,36.2,2013/11/3,Scheduled visit,No,0.0,74.0,<37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48717,349720999,3497,HH112000801,No,0.0,NaN,1.32,No,0.0,0.0,...,NaN,No,Negative,36.4,2017/7/6,Enrollment,No,0.0,8.0,<37
48718,349820999,3498,HH118002402,No,0.0,NaN,1.61,No,0.0,0.0,...,NaN,No,Negative,37.0,2017/6/22,Enrollment,No,0.0,10.0,<37
48719,349920999,3499,HH145001701,No,0.0,NaN,2.08,No,0.0,0.0,...,NaN,No,Negative,37.0,2017/6/30,Enrollment,No,0.0,10.0,<37
48720,350020999,3500,HH116000604,No,0.0,NaN,1.32,No,0.0,0.0,...,NaN,No,Negative,37.2,2017/6/28,Enrollment,No,0.0,10.0,37-38


In [2]:
#Identify the type of each column of data -'int64', 'object' or 'float64' - and store it as an array,
#'observ_type_array'
observ_index = observations.keys()
observ_index_list = observ_index.tolist()
observ_index_array = np.array(observ_index_list)
observ_type_array = [0]*54
for i in range(observ_index_array.size):
    name = observ_index_array[i]
    temp = str(observations[name].dtype)
    observ_type_array[i]=temp
observ_type_array = np.array(observ_type_array)

In [8]:
# Pick columns consisting of numerical data and store the values as a 48622*19 array 'float_values'
#so the volume of data is 48622, the dimension of data is 19
float_position = np.where(observ_type_array == 'float64')[0]
float_observ = observations.iloc[:,float_position]
float_values = float_observ.fillna(0).values

In [41]:
# Randomly pick two subsets of the data, each of size 10,000
import random
random_vector_1 = [random.randint(0,48721) for i in range(10000)]
random_vector_2 = [random.randint(0,48721) for i in range(10000)]

In [42]:
#Apply t-SNE to the two subsets
from sklearn.manifold import TSNE
pdf_embedded_1 = TSNE(n_components=2).fit_transform(float_values[random_vector_1])
pdf_embedded_2 = TSNE(n_components=2).fit_transform(float_values[random_vector_2])

In [68]:
#I generated a random data of same volume and dimension, to see whether the distribution would be distinc
#as expected
rnd = np.random.rand(10000,19)
rnd_embedded = TSNE(n_components=2).fit_transform(rnd)

In [45]:
#I want to differentiate a subset of data (the input random_vector) with 'yes' and 'no' to malaria, 
#in order to see why they are distributed and clustered in different parts of the t-SNE graph
def malaria_diff(random_vector):
    malaria_column = observations.loc[random_vector,'Malaria diagnosis [EUPATH_0000090]'].values
    malaria_yes = np.where(malaria_column == 'Yes')
    malaria_no = np.where(malaria_column == 'No')
    return malaria_yes,malaria_no


((array([   4,    7,    9, ..., 9977, 9979, 9989]),),
 (array([   0,    1,    2, ..., 9997, 9998, 9999]),))

In [11]:
malaria_yes_1,malaria_no_1 = malaria_diff(random_vector_1)
malaria_yes_2,malaria_no_2 = malaria_diff(random_vector_2)
x1 = pdf_embedded_1[malaria_yes_1]
y1 = pdf_embedded_1[malaria_no_1]
x2 = pdf_embedded_2[malaria_yes_2]
y2 = pdf_embedded_2[malaria_no_2]


#You can play around the commented lines to do different things, like plotting two graphs and compare, or plotting
#that random data set and see the difference.
plt.figure(1)

plt.scatter(x1[:,0],x1[:,1], s=10, c='b', marker="s", label='yes1',alpha=1)
plt.scatter(y1[:,0],y1[:,1], s=10, c='k', marker="o", label='no1',alpha=0.1)
# plt.show()

# plt.figure(2)
# ax2 = fig.add_subplot(212)
plt.scatter(x2[:,0],x2[:,1], s=10, c='y', marker="s", label='yes2',alpha=1)
plt.scatter(y2[:,0],y2[:,1], s=10, c='r', marker="o", label='no2',alpha=0.1)
# plt.legend(loc='upper left');

# plt.scatter(rnd_embedded[:,0],rnd_embedded[:,1], s=10, c='r', marker="s", label='yes2',alpha=0.1)
plt.show()

NameError: name 'malaria_diff' is not defined